# 4.3 Review Features: Votes

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import RobustScaler

In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords
languages = [
    'arabic', 'azerbaijani', 'bengali', 'danish', 'dutch', 'english',
    'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian',
    'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian',
    'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish'
]
english_stopwords = set(stopwords.words('english'))
all_stopwords = set()
for language in languages:
    all_stopwords |= set(stopwords.words(language))

[nltk_data] Downloading package stopwords to C:\Users\Conor Mac
[nltk_data]     Amhlaoibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
DIR_SAMPLE = '../data/samples/'
PATH_REVSENTS = DIR_SAMPLE + 'review_sentiment/%s.csv'

In [4]:
def preprocess_text(text, is_english=True):
    if is_english:
        text = text.lower()
        stopword_list = english_stopwords
    else:
        stopword_list = all_stopwords
    text = re.sub(r'[^\w\s]', ' ', text, re.UNICODE)
    text = ' '.join([
        word
        for word in text.split()
        if word not in stopword_list
    ])
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
def read_data(sample_name, train_size=0.8, seed=None):
    df = pd.read_csv(PATH_REVSENTS % sample_name)[['votes_up', 'text']].reset_index(drop=True)
    if sample_name.startswith('eng'):
        df['text'] = df['text'].apply(lambda text: preprocess_text(text, is_english=True))
    else:
        df['text'] = df['text'].apply(lambda text: preprocess_text(text, is_english=False))
    scaler = RobustScaler()
    df['votes_up_scaled'] = scaler.fit_transform(np.array(df['votes_up']).reshape(-1, 1))
    train, validation = train_test_split(df, train_size=train_size, random_state=seed)
    return train, validation

In [6]:
train, validation = read_data('eng_any_any_100000')
Xt, Yt = train['text'].tolist(), train['votes_up_scaled'].tolist()
Xv, Yv = validation['text'].tolist(), validation['votes_up_scaled'].tolist()

In [7]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), binary=True, smooth_idf=False)
Xt_tfidf = tfidf.fit_transform(Xt)
Xv_tfidf = tfidf.transform(Xv)

In [9]:
alphas = [1]#[0.1, 1, 10]
for alpha in alphas:
    bl_model1 = DummyRegressor(strategy='mean')
    bl_model2 = DummyRegressor(strategy='constant', constant=-0.5)
    model = SGDRegressor(max_iter=500)
    bl_mse1 = cross_val_score(bl_model1, Xt_tfidf, Yt, scoring='neg_mean_squared_error', cv=5)
    bl_mse2 = cross_val_score(bl_model2, Xt_tfidf, Yt, scoring='neg_mean_squared_error', cv=5)
    mse = cross_val_score(model, Xt_tfidf, Yt, scoring='neg_mean_squared_error', cv=5)
    print(mse.mean())
    print(bl_mse1.mean())
    print(bl_mse2.mean())

C:\Users\Conor Mac Amhlaoibh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Conor Mac Amhlaoibh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Conor Mac Amhlaoibh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\Users\Conor Mac Amhlaoibh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1211: Convergence

-118.52328382904616
-120.74512672451172
-126.17490312499999


C:\Users\Conor Mac Amhlaoibh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [93]:
model = MultinomialNB(alpha=1)
model.fit(Xt_tfidf, Yt)
probs = model.predict_proba(Xv_tfidf)
Yp = np.where(probs[:,1] >= 0.5, 1, 0).tolist()
print(accuracy_score(Yv, Yp))

0.615


In [94]:
model = ComplementNB(alpha=1)
model.fit(Xt_tfidf, Yt)
probs = model.predict_proba(Xv_tfidf)
Yp = np.where(probs[:,1] >= 0.5, 1, 0).tolist()
print(accuracy_score(Yv, Yp))

0.615


In [96]:
model = SGDClassifier(loss='hinge', penalty='l2', max_iter=20)
model.fit(Xt_tfidf, Yt)
Yp = model.predict(Xv_tfidf).tolist()
print(accuracy_score(Yv, Yp))

0.67
